# Demo Scenario 3
> Validate Demonstrate Scenario 3 on a notebook

In [1]:
#| hide
from nbdev.showdoc import *

In [2]:
#| eval: false

from llmcam.downloader import *
from llmcam.dtcam import *
from llmcam.file_manager import *
from llmcam.fn_to_fc import *
from llmcam.oas_to_requests import *
from llmcam.store import *
from llmcam.yolo import *

In [4]:
def update_tools(
    services: list # list of services
):
    """Extarct services from ToolBox and integrate to current tools"""
    new = extract_tools_from_services(services)
    for tool in new:
        tools.append(tool)

tools = [tool_schema(fn) for fn in [update_tools, download_file, cap, add_api_tools]] + extract_tools_from_services(['ytube_live'])
tools[0]["function"]["parameters"]["properties"]["services"] = {
            "type": "array",
            "description": "List of service names",
            "items": {"type": "string"}
        }


In [5]:
messages = form_msgs([
    ("system", "You are a helpful system administrator. Use the supplied tools to help the user."),
    ("user", """Notify me when there is a vacancy in the parking lot. Check these cameras:
            https://youtu.be/SUseaJu9DIE?si=LFM0yN-VhS5JJACO
            https://youtu.be/bK26jwnbyPw?si=7zn_yAnYLq57dMmX
            https://youtu.be/8jBwkS2CLWQ?si=rnEPCAHcQ11iaog0
"""),
])
complete(messages, tools)
print_msgs(messages)

[youtube] Extracting URL: https://youtu.be/SUseaJu9DIE?si=LFM0yN-VhS5JJACO
[youtube] SUseaJu9DIE: Downloading webpage
[youtube] SUseaJu9DIE: Downloading ios player API JSON
[youtube] SUseaJu9DIE: Downloading mweb player API JSON
[youtube] SUseaJu9DIE: Downloading m3u8 information
[youtube] Extracting URL: https://youtu.be/bK26jwnbyPw?si=7zn_yAnYLq57dMmX
[youtube] bK26jwnbyPw: Downloading webpage
[youtube] bK26jwnbyPw: Downloading ios player API JSON
[youtube] bK26jwnbyPw: Downloading mweb player API JSON
[youtube] bK26jwnbyPw: Downloading m3u8 information
[youtube] Extracting URL: https://youtu.be/8jBwkS2CLWQ?si=rnEPCAHcQ11iaog0
[youtube] 8jBwkS2CLWQ: Downloading webpage
[youtube] 8jBwkS2CLWQ: Downloading ios player API JSON
[youtube] 8jBwkS2CLWQ: Downloading mweb player API JSON
[youtube] 8jBwkS2CLWQ: Downloading m3u8 information
>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
Notify me when there is a vacancy in the parking lot. Ch

1. User: Notify me when there’s a vacancy in the parking lot.
2. Assistant: Notification set. I’ll alert you as soon as a parking spot becomes available.
3. *(Later, when a space opens)* Assistant: Notification: A parking spot is now available at [Location].

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()